In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
%cd /content/detectron2
!pip install -r requirements.txt
!pip install -e .
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install optuna

Cloning into 'detectron2'...
remote: Enumerating objects: 15258, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 15258 (delta 147), reused 155 (delta 78), pack-reused 14979
Receiving objects: 100% (15258/15258), 6.28 MiB | 26.25 MiB/s, done.
Resolving deltas: 100% (11009/11009), done.
/content/detectron2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.0 MB/s eta 0:00:00
  Prep

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Bangkok /etc/localtime
!date

Mon Sep 25 11:18:47 AM +07 2023


In [ ]:
import sys
import os
import time
import logging
import numpy as np
import cv2
import torch
import datetime

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultTrainer
from detectron2.modeling import build_model
from detectron2.modeling import BACKBONE_REGISTRY, Backbone, ShapeSpec
from torchvision.models import mobilenet_v2

# 1. เชื่อมต่อกับ Google Drive
from google.colab import drive
drive.mount('/content/drive')

from detectron2.data.datasets import register_coco_instances
# 2. ปรับเปลี่ยน directory paths
BASE_PATH = "/content/drive/MyDrive/laboro_tomato" # กรุณาเปลี่ยนเป็น path ของคุณ
register_coco_instances("tomato_train", {}, f"{BASE_PATH}/annotations/train.json", f"{BASE_PATH}/train")
register_coco_instances("tomato_test", {}, f"{BASE_PATH}/annotations/test.json", f"{BASE_PATH}/test")


Mounted at /content/drive


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

train_metadata = MetadataCatalog.get("tomato_train")
train_dataset_dicts = DatasetCatalog.get("tomato_train")
test_metadata = MetadataCatalog.get("tomato_test")
test_dataset_dicts = DatasetCatalog.get("tomato_test")

import optuna

[09/25 07:37:43 d2.data.datasets.coco]: Loading /content/drive/MyDrive/laboro_tomato/annotations/train.json takes 1.59 seconds.
[09/25 07:37:43 d2.data.datasets.coco]: Loaded 643 images in COCO format from /content/drive/MyDrive/laboro_tomato/annotations/train.json
[09/25 07:37:44 d2.data.datasets.coco]: Loaded 161 images in COCO format from /content/drive/MyDrive/laboro_tomato/annotations/test.json


In [ ]:
# import random
# for d in random.sample(train_dataset_dicts, 3):
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
#     vis = visualizer.draw_dataset_dict(d)
#     cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
MODEL_BASE = "COCO-InstanceSegmentation"
MODEL_USED = "mask_rcnn_R_101_C4_3x.yaml"
EPOCH = 100
INPUT_SIZE = (640,)
DEVICE = "cuda"
MODEL_INPUT = f"{MODEL_BASE}/{MODEL_USED}"
logger = logging.getLogger()

def setup_directories(model_used):
    base_output_dir = os.path.join("..", "train", model_used)
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    specific_output_dir = os.path.join(base_output_dir, current_time)

    before_test_dir = os.path.join(specific_output_dir, "before")
    after_test_dir = os.path.join(specific_output_dir, "after")
    os.makedirs(before_test_dir, exist_ok=True)
    os.makedirs(after_test_dir, exist_ok=True)

    log_dir = os.path.join(specific_output_dir)
    os.makedirs(log_dir, exist_ok=True)

    log_file_path = setup_logging(log_dir)
    return before_test_dir, after_test_dir, log_file_path


def setup_logging(log_dir, log_filename="training.log"):
    log_file_path = os.path.join(log_dir, log_filename)

    for handler in logger.handlers[:]:
        logger.removeHandler(handler)

    file_handler = logging.FileHandler(log_file_path)
    stream_handler = logging.StreamHandler() # สร้าง StreamHandler
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    file_handler.setFormatter(formatter)
    stream_handler.setFormatter(formatter) # กำหนด formatter ให้กับ StreamHandler

    logger.addHandler(file_handler)
    logger.addHandler(stream_handler) # เพิ่ม StreamHandler ไปยัง logger
    logger.setLevel(logging.INFO)

    return log_file_path


def configure_model(model_input, lr, epoch, input_size, dv):
    cfg = get_cfg()
    cfg.OUTPUT_DIR = "/content/drive/MyDrive/Detectron2_model"
    cfg.merge_from_file(model_zoo.get_config_file(model_input))
    cfg.DATASETS.TRAIN = ("tomato_train",)
    cfg.DATASETS.TEST = ("tomato_test",)
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_input)
    cfg.SOLVER.IMS_PER_BATCH = 4
    # cfg.SOLVER.STEPS = (128,176)
    cfg.INPUT.FORMAT = "RGB"
    cfg.SOLVER.WEIGHT_DECAY = wd
    cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE = "norm"
    cfg.SOLVER.WEIGHT_DECAY_BIAS = wd
    cfg.SOLVER.BASE_LR = lr
    cfg.SOLVER.MAX_ITER = epoch
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32, 64, 128, 256, 512]]
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
    cfg.INPUT.MIN_SIZE_TRAIN = input_size
    cfg.MODEL.DEVICE = dv
    return cfg


def save_config(cfg, model_base, model_used):
    config_directory = f'./train/config/{model_base}/'
    os.makedirs(config_directory, exist_ok=True)
    config_path = os.path.join(config_directory, model_used)
    with open(config_path, 'w') as f:
        f.write(cfg.dump())


def save_model_summary(model, model_base, model_used):
    model_summary_directory = f'./train/model_summary/{model_base}/'
    os.makedirs(model_summary_directory, exist_ok=True)
    model_summary_path = os.path.join(model_summary_directory, model_used.replace('.yaml', '.txt'))
    with open(model_summary_path, 'w') as f:
        print(str(model), file=f)


class Trainer(DefaultTrainer):
    before_test_dir = None

    def __init__(self, cfg, before_test_dir=None):
        super(Trainer, self).__init__(cfg)
        Trainer.before_test_dir = before_test_dir

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = cls.before_test_dir
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


def train_model(cfg, before_test_dir):
    trainer = Trainer(cfg, before_test_dir=before_test_dir)
    trainer.resume_or_load(resume=False)
    trainer.train()

# ค่าที่ Optuna หามาจากการเทรน YOLOv8
best_params = {
    'lr0': 0.04088758317998441,
    'lrf': 0.010560736792910063,
    'momentum': 0.647009849179402,
    'weight_decay': 2.283225438506775e-05,
    'warmup_epochs': 5,
    'warmup_momentum': 0.17887156211826807
}

def configure_model_for_detectron2(best_params):
    bz = 2
    cfg = get_cfg()
    cfg.OUTPUT_DIR = "/content/drive/MyDrive/Detectron2_model"
    cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml")
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL_INPUT)
    cfg.DATASETS.TRAIN = ("tomato_train",)
    cfg.DATASETS.TEST = ("tomato_test",)
    cfg.SOLVER.IMS_PER_BATCH = bz
    cfg.DATALOADER.NUM_WORKERS = 4
    # ใช้ค่า best_params จาก Optuna ในการปรับแต่ง cfg
    cfg.SOLVER.MAX_ITER = 3300
    cfg.SOLVER.BASE_LR = best_params['lr0']
    cfg.SOLVER.BASE_LR_END =  best_params['lrf']
    cfg.SOLVER.WEIGHT_DECAY = best_params['weight_decay']
    cfg.SOLVER.MOMENTUM = best_params['momentum']
    cfg.SOLVER.WARMUP_ITERS =  best_params['warmup_epochs']*(643/bz)
    cfg.SOLVER.WARMUP_FACTOR = best_params['warmup_momentum']
    cfg.SOLVER.WARMUP_START_LR = best_params['lr0'] * best_params['warmup_momentum']
    cfg.MODEL.DEVICE = (DEVICE)
    cfg.INPUT.FORMAT = "RGB"
    cfg.INPUT.MIN_SIZE_TRAIN = (640,)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
    # ... คุณสามารถตั้งค่าพารามิเตอร์อื่น ๆ ที่คุณต้องการให้เป็นค่าจาก best_params
    return cfg

# ปรับแต่ง Detectron2 config ด้วย best_params
cfg = configure_model_for_detectron2(best_params)


def main():
    before_test_dir, after_test_dir, log_file_path = setup_directories(MODEL_USED)
    cfg = configure_model_for_detectron2(best_params)
    save_config(cfg, MODEL_BASE, MODEL_USED)

    model = build_model(cfg)
    save_model_summary(model, MODEL_BASE, MODEL_USED)

    start_time = time.time()
    train_model(cfg, before_test_dir)
    end_time = time.time()

    # with torch.no_grad():
    #     evaluator = COCOEvaluator("tomato_test", cfg, False, after_test_dir)
    #     val_loader = build_detection_test_loader(cfg, "tomato_test")
    #     inference_on_dataset(model, val_loader, evaluator)

    total_time = end_time - start_time
    minutes = total_time // 60
    seconds = total_time % 60
    print(best_params)

if __name__ == '__main__':
    torch.cuda.empty_cache()
    main()
    torch.cuda.empty_cache()

[09/25 07:38:22 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

2023-09-25 07:38:22,358 - INFO - Downloading https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x/138363239/model_final_a2914c.pkl ...
2023-09-25 07:38:22,359 - INFO - Downloading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x/138363239/model_final_a2914c.pkl ...
model_final_a2914c.pkl: 220MB [00:15, 14.6MB/s]                           
2023-09-25 07:38:37,487 - INFO - Successfully downloaded /root/.torch/iopath_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x/138363239/model_final_a2914c.pkl. 220308676 bytes.
2023-09-25 07:38:37,488 - INFO - URL https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x/138363239/model_final_a2914c.pkl cached in /root/.torch/iopath_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x/138363239/model_final_a2914c.pkl
2023-09-25 07:38:37,490 - INFO - [Checkpointer] Loading from /root/.torch/iopath_cache/detectro

[09/25 07:38:37 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/25 07:39:19 d2.utils.events]:  eta: 1:27:18  iter: 19  total_loss: 1.689  loss_cls: 0.5397  loss_box_reg: 0.697  loss_mask: 0.3835  loss_rpn_cls: 0.0511  loss_rpn_loc: 0.02887    time: 1.5747  last_time: 1.6273  data_time: 0.3113  last_data_time: 0.0040   lr: 0.0077105  max_mem: 8940M


2023-09-25 07:39:22,953 - INFO - NumExpr defaulting to 8 threads.


[09/25 07:39:56 d2.utils.events]:  eta: 1:27:46  iter: 39  total_loss: 1.266  loss_cls: 0.5223  loss_box_reg: 0.4991  loss_mask: 0.1943  loss_rpn_cls: 0.0217  loss_rpn_loc: 0.02729    time: 1.6167  last_time: 1.7160  data_time: 0.0047  last_data_time: 0.0045   lr: 0.0081282  max_mem: 8940M
[09/25 07:40:30 d2.utils.events]:  eta: 1:30:08  iter: 59  total_loss: 1.139  loss_cls: 0.5057  loss_box_reg: 0.3739  loss_mask: 0.1798  loss_rpn_cls: 0.02545  loss_rpn_loc: 0.02126    time: 1.6519  last_time: 1.7170  data_time: 0.0047  last_data_time: 0.0050   lr: 0.0085459  max_mem: 8940M
[09/25 07:41:05 d2.utils.events]:  eta: 1:30:28  iter: 79  total_loss: 1.063  loss_cls: 0.4678  loss_box_reg: 0.3272  loss_mask: 0.1775  loss_rpn_cls: 0.04651  loss_rpn_loc: 0.03345    time: 1.6665  last_time: 1.6998  data_time: 0.0054  last_data_time: 0.0051   lr: 0.0089636  max_mem: 8940M
[09/25 07:41:39 d2.utils.events]:  eta: 1:30:01  iter: 99  total_loss: 0.9101  loss_cls: 0.3596  loss_box_reg: 0.2959  loss_m

2023-09-25 09:11:45,399 - INFO - Saving checkpoint to /content/drive/MyDrive/Detectron2_model/model_final.pth


[09/25 09:11:56 d2.utils.events]:  eta: 0:00:00  iter: 3299  total_loss: 0.3567  loss_cls: 0.08431  loss_box_reg: 0.1567  loss_mask: 0.08197  loss_rpn_cls: 0.003816  loss_rpn_loc: 0.02056    time: 1.6886  last_time: 1.7172  data_time: 0.0048  last_data_time: 0.0048   lr: 0.040888  max_mem: 8942M
[09/25 09:11:56 d2.engine.hooks]: Overall training speed: 3298 iterations in 1:32:48 (1.6886 s / it)
[09/25 09:11:56 d2.engine.hooks]: Total training time: 1:33:05 (0:00:16 on hooks)
[09/25 09:11:56 d2.data.datasets.coco]: Loaded 161 images in COCO format from /content/drive/MyDrive/laboro_tomato/annotations/test.json
[09/25 09:11:56 d2.data.build]: Distribution of instances among all 6 categories:
|   category    | #instances   |   category    | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
| b_fully_rip.. | 72           | b_half_ripe.. | 116          |  b_green   | 387          |
| l_fully_rip.. | 269    

In [ ]:
(65.73+66.33)/2

66.03

In [ ]:
# MODEL_BASE = "COCO-InstanceSegmentation"
# MODEL_USED = "mask_rcnn_R_50_C4_3x.yaml"
# EPOCH = 2900
# INPUT_SIZE = (672,672)
# DEVICE = "cuda"
# MODEL_INPUT = f"{MODEL_BASE}/{MODEL_USED}"

# def objective(trial):
#     lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
#     wd = trial.suggest_float("weight_decay", 1e-4, 1e-3)

#     cfg = configure_model(MODEL_INPUT, lr, EPOCH, INPUT_SIZE, DEVICE)
#     trainer = Trainer(cfg)
#     trainer.resume_or_load(resume=False)
#     trainer.train()

#     return 1.0  # Dummy value

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print(f"Best hyperparameters: {study.best_params}")


In [ ]:
import yaml
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
cfg = configure_model(MODEL_INPUT, LR, EPOCH, INPUT_SIZE, DEVICE)
config_yaml_path = "/content/drive/MyDrive/Detectron2_model/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/Detectron2_model/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[09/14 17:45:31 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/Detectron2_model/model_final.pth ...


2023-09-14 17:45:31,926 - INFO - [Checkpointer] Loading from /content/drive/MyDrive/Detectron2_model/model_final.pth ...


In [ ]:
# from detectron2.utils.visualizer import ColorMode
# import random

# for d in random.sample(test_dataset_dicts, 1):    #select number of images for display
#     im = cv2.imread(d["file_name"])
#     outputs = predictor(im)
#     v = Visualizer(im[:, :, ::-1],
#                    metadata=test_metadata,
#                    scale=0.5,
#                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
#     )
#     out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#     cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# new_im = cv2.imread("/content/drive/MyDrive/laboro_tomato/train/IMG_0997.jpg")
# outputs  = predictor(new_im)

# # We can use `Visualizer` to draw the predictions on the image.
# v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata)
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
import os
import shutil

def copy_log_files(source_base_dir, dest_base_dir):
    """
    Copy .log files from source_base_dir to dest_base_dir.

    Args:
    - source_base_dir: The source directory (e.g., 'train\mask_rcnn_R_50_C4_1x.yaml')
    - dest_base_dir: The destination directory (e.g., 'log')
    """
    # เข้าไปยัง source_base_dir และวนซ้ำสำหรับแต่ละไดเรกทอรีที่อยู่ภายใน
    for subdir in os.listdir(source_base_dir):
        source_subdir_path = os.path.join(source_base_dir, subdir)

        # ตรวจสอบว่าเป็นไดเรกทอรีหรือไม่
        if os.path.isdir(source_subdir_path):
            log_file_path = os.path.join(source_subdir_path, 'training.log')

            # ตรวจสอบว่ามีไฟล์ training.log ในไดเรกทอรีนั้นหรือไม่
            if os.path.exists(log_file_path):
                dest_subdir_path = os.path.join(dest_base_dir, subdir)
                dest_log_file_path = os.path.join(dest_subdir_path, 'training.log')

                # สร้างไดเรกทอรีเป้าหมายถ้ายังไม่มี
                os.makedirs(dest_subdir_path, exist_ok=True)

                # คัดลอกไฟล์ .log ไปยังไดเรกทอรีเป้าหมาย
                shutil.copy2(log_file_path, dest_log_file_path)
                print(f"Copied {log_file_path} to {dest_log_file_path}")
# ตัวอย่างการเรียกใช้
model="mask_rcnn_R_50_C4_1x"
source_dir = f"/content/train/{model}.yaml"
destination_dir = f"../log/{model}/"
copy_log_files(source_dir, destination_dir)


Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-131746/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-131746/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-154440/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-154440/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-135023/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-135023/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-145436/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-145436/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-142209/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-142209/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-165416/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-165416/training.log
Copied /content/train/mask_rcnn_R_50_C4_1x.yaml/20230913-052047/training.log to ../log/mask_rcnn_R_50_C4_1x/20230913-052047/training.log
Copied /content/train/mask_rcnn_R_50_C4_1

In [ ]:
import os
import glob

# Path ของโฟลเดอร์หลัก
base_path = "/content/log"
output_base_path = "/content/data_train"


def extract_info_from_log(file_path):
    with open(file_path, "r", encoding='utf-8') as f:  # ใส่ encoding='utf-8' ที่นี่
        lines = f.readlines()

    extract_data = []
    skip_mode = False  # True if we are currently skipping lines
    for line in lines:
        # Check for start lines
        if (
            "Total training time:" in line
            or "Total inference time:" in line
            or "Evaluation results for bbox:" in line
            or "Evaluation results for segm:" in line
            or "copypaste: Task: bbox" in line
        ):
            skip_mode = True
            extract_data.append(line.strip())
        elif skip_mode:
            # Check for stop lines
            if (
                "Loaded 161 images in COCO format" in line
                or "Preparing results for COCO format" in line
                or "Evaluate annotation type *segm*" in line
                or "Evaluation results for tomato_test in csv format:" in line
            ):
                skip_mode = False
            else:
                extract_data.append(line.strip())

    return extract_data


def save_extracted_data_to_file(extracted_data, output_file_path):
    with open(output_file_path, "w", encoding='utf-8') as f:  # กำหนด encoding ที่นี่
        for data in extracted_data:
            f.write(data + "\n")


# ค้นหาไฟล์ log ทั้งหมด
for model_folder in glob.glob(os.path.join(base_path, "mask_rcnn_*")):
    for log_file in glob.glob(os.path.join(model_folder, "*/*training.log")):
        # สกัดข้อมูล
        extracted_data = extract_info_from_log(log_file)

        # สร้าง path สำหรับบันทึกข้อมูล
        rel_path = os.path.relpath(log_file, base_path)
        output_file_path = os.path.join(output_base_path, rel_path)

        # สร้างโฟลเดอร์ถ้ายังไม่มี
        output_dir = os.path.dirname(output_file_path)
        os.makedirs(output_dir, exist_ok=True)

        # บันทึกข้อมูล
        save_extracted_data_to_file(extracted_data, output_file_path)

In [ ]:
import shutil
shutil.make_archive('data_train', 'zip', '/content/data_train')
from google.colab import files
files.download('data_train.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>